In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier

# Prova Itaú - Classificação

# Prova 1.

## Questão 3) 

Considerando os dados presentes no arquivo class01.csv, treine o algoritmo Naive Bayes Gaussiano utilizando a metodologia de validação cruzada holdout (utilize para treino as 350 primeiras linhas e para validação as demais). Qual o valor da acurácia para a base de treino? Qual o valor da acurácia para a base de validação? Faça o mesmo treinamento com a metodologia Leave-One-Out. Qual o valor da acurácia média para a base de treino? Qual o valor da acurácia média para a base de validação?

In [2]:
q3 = pd.read_csv('data/asset-v1_IRONHACK+DAFT+202002_SAO+type@asset+block@class01.csv')
q3.head(3)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,target
0,1.162990,-0.831971,-0.185266,0.441240,0.033957,1.432988,-1.171483,0.203248,1.900889,1.119174,...,-1.139286,-0.608239,-0.107224,1.700972,-0.698151,0.520004,-0.450784,-0.187694,-0.538329,1.0
1,-1.750976,-0.516614,-0.761588,1.692347,-1.579436,0.581782,-0.950953,0.381426,-1.370027,0.943387,...,-0.280502,-0.306014,0.554926,-1.402866,-0.776478,1.181890,-0.066449,-0.802430,1.587229,0.0
2,0.304635,0.972369,0.202908,-0.607761,0.487088,0.205173,0.815216,0.491880,0.904705,1.167017,...,-0.134901,0.291496,-1.038477,1.297057,-1.296614,-1.109806,-0.003495,-0.149940,0.539892,8.0


In [3]:
q3.shape

(1000, 101)

## Hold Out

In [4]:
q3_train = q3.iloc[:350]
q3_test = q3.iloc[350:]

In [5]:
q3_X_train = q3_train.drop(['target'], axis=1)
q3_y_train = q3_train['target']
q3_X_test = q3_test.drop(['target'], axis=1)
q3_y_test = q3_test['target']

In [6]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(q3_X_train, q3_y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [7]:
print(f"Train score: {clf.score(q3_X_train, q3_y_train)}")
print(f"Test score: {clf.score(q3_X_test, q3_y_test)}")

Train score: 0.76
Test score: 0.6276923076923077


## Leave One Out

In [8]:
from sklearn.model_selection import LeaveOneOut

In [9]:
X_q3 = q3.drop(['target'], axis=1)
y_q3 = q3['target']

In [10]:
loo = LeaveOneOut()
loo.get_n_splits(X_q3)

1000

In [13]:
train_score = []
test_score = []

for train_index, test_index in loo.split(X_q3):
    X_q3_train, X_q3_test = X_q3.iloc[train_index], X_q3.iloc[test_index]
    y_q3_train, y_q3_test = y_q3.iloc[train_index], y_q3.iloc[test_index]

    model = GaussianNB()
    model.fit(X_q3_train, y_q3_train)
    train_res = model.score(X_q3_train, y_q3_train)
    test_res = model.score(X_q3_test, y_q3_test)
    train_score.append(train_res)
    test_score.append(test_res)

print(f"Train: {np.mean(train_score)}")
print(f"Test: {np.mean(test_score)}")

Train: 0.6892802802802803
Test: 0.642


In [14]:
from sklearn.model_selection import cross_val_score, KFold, cross_validate

In [15]:
model = GaussianNB()

scores = cross_val_score(model, X_q3, y_q3, cv=KFold(n_splits=X_q3.shape[0]))

In [16]:
print(f"Mean Score with KFold: {scores.mean()}")

Mean Score with KFold: 0.642


In [17]:
scores2 = cross_validate(model, X_q3, y_q3, cv=KFold(n_splits=X_q3.shape[0]), return_train_score=True)

In [18]:
print(f"Mean Score with KFold: {scores2['train_score'].mean()}")
print(f"Mean Score with KFold: {scores2['test_score'].mean()}")

Mean Score with KFold: 0.6892802802802803
Mean Score with KFold: 0.642


## Questão 4) 

Considerando os dados presentes no arquivo class02.csv, treine o algoritmo 10-Nearest Neighbors (KNN com 𝑘=10 e distância Euclidiana), utilizando a metodologia de validação cruzada k-fold com 10 folds não estratificados. Considere que a primeira pasta de validação seja formada pelas primeiras 10% linhas do arquivo, que a segunda pasta de validação seja formada pelas 10% linhas seguintes, e assim por diante, até atingir a última pasta, formada pelas 10% linhas finais da base. Qual o valor médio da acurácia para a base de validação?

In [69]:
q4 = pd.read_csv('data/asset-v1_IRONHACK+DAFT+202002_SAO+type@asset+block@class02.csv')
q4.head(3)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,target
0,1.753916,-1.283809,-0.474435,-0.328502,1.162749,1.110096,-0.532774,1.041946,0.393644,1.508894,...,-0.102623,0.233644,-1.711628,0.816312,2.700492,2.813359,0.343391,0.348734,-0.269752,5.0
1,0.579139,0.499696,1.848421,-2.037663,-0.490755,-0.874975,0.988246,0.117195,0.641343,1.041325,...,-0.113718,0.918633,1.212413,2.534996,-1.694515,-2.519081,0.531062,-0.429925,-1.033625,2.0
2,3.455631,-0.843593,-1.180263,-1.388971,-0.763881,-0.809791,-0.225052,0.818787,1.021153,-0.486736,...,-0.812184,-0.677958,2.545184,2.228583,-1.911287,-3.965635,1.293003,0.196379,-0.739823,3.0


In [70]:
q4.shape
X_q4 = q4.drop(['target'], axis=1)
y_q4 = q4['target']

In [71]:
## precisa standarizar o X para KNN
from sklearn.preprocessing import StandardScaler
knn = KNeighborsClassifier(n_neighbors=10, p=2, metric='minkowski')
result = cross_val_score(knn, StandardScaler().fit_transform(X_q4), y_q4, cv=KFold(10))

In [72]:
print(f"Mean score: {np.mean(result)}")

Mean score: 0.758


In [73]:
knn2 = KNeighborsClassifier(n_neighbors=10, p=2, metric='minkowski')
result2 = cross_validate(knn2, StandardScaler().fit_transform(X_q4), y_q4, cv=KFold(10), return_train_score=True)

In [74]:
print(f"Mean Score with KFold: {result2['train_score'].mean()}")
print(f"Mean Score with KFold: {result2['test_score'].mean()}")

Mean Score with KFold: 0.8204444444444444
Mean Score with KFold: 0.758


## Questão 5 

Assinale as alternativas com V ou F para Verdadeiro ou Falso respectivamente. Atente para o fato que uma questão errada anula uma certa. Em caso de dúvidas deixe em branco.

(**F**) No método KNN, o melhor valor de 𝑘 é igual a 𝑛 (𝑛 = número de exemplos), porém dado o custo computacional, valores pequenos de 𝑘 são preferidos.

(**F**) As folhas de uma Árvore de Decisão são obtidas objetivando-se minimizar a pureza entre classes.

(**V**) Uma Árvore de Decisão completa (profundidade máxima possível) tem maior chance de fazer overfitting nos dados do que uma árvore com profundidade limitada.

( ) Para utilizar uma rede neural em um problema multiclasse é necessário o uso de estratégias como One-Vs-One e One-Vs-Rest.

(**F**) Root Mean Squared Error (RMSE) é uma medida adequada para avaliar classificadores.

(**V**) Todo algoritmo de classificação possui um viés que dita a forma como cada método explora o espaço de busca pela hipótese que melhor se ajusta aos dados.

(**F**) O grau do polinômio do algoritmo Support Vector Machine (SVM) deve ser definido para todos os kernels.

() Uma vantagem dos algoritmos baseados em árvores é que, em sua maioria, a presença de valores faltantes (missing) não inviabiliza sua execução.

<u>Resposta meio dúbia, pode ser tanto Verdadeira quanto Falsa</u>

(**F**) Utiliza-se validação out-of-time separando um conjunto de dados de forma aleatória independentemente do tempo.

(**V**) O Algoritmo MLP (Multi-Layer Perceptron) é conceitualmente baseado em uma visão abstrata e simplificada de um neurônio biológico

-----

# Prova 2.

## Questão 3

Considerando os dados presentes no arquivo classificacao_Q3.csv, treine o algoritmo Naive Bayes utilizando a metodologia de validação cruzada holdout (utilize para treino as 500 primeiras linhas e para validação as demais) Qual o valor da acurácia a base de treino? Qual o valor da acurácia média para as bases de validação?

In [23]:
q5 = pd.read_csv('data/asset-v1_IRONHACK+DAFT+202002_SAO+type@asset+block@classificacao_Q3.csv')
q5.head(3)

,Genero,Idade,Escolaridade,Profissao,Target
0,F,a - Ate 25 anos,Fundamental,b,0
1,M,a - Ate 25 anos,Medio,d,1
2,F,c - 36 a 45 anos,Fundamental,a,1


In [24]:
q5.dtypes

Genero          object
Idade           object
Escolaridade    object
Profissao       object
Target           int64
dtype: object

In [25]:
q5.shape

(1000, 5)

In [26]:
X_q5 = q5.drop(['Target'], axis=1)
y_q5 = q5['Target']

In [27]:
X_q5_train = X_q5[:500]
y_q5_train = y_q5[:500]
X_q5_test = X_q5[500:]
y_q5_test = y_q5[500:]

In [28]:
from sklearn.naive_bayes import ComplementNB, MultinomialNB, CategoricalNB, BernoulliNB

In [38]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X_q5_train_std = enc.fit_transform(X_q5_train)
X_q5_test_std = enc.fit_transform(X_q5_test)

In [39]:
X_q5_std

array([[0., 0., 0., 1.],
       [1., 0., 1., 3.],
       [0., 2., 0., 0.],
       ...,
       [1., 1., 0., 0.],
       [1., 2., 1., 1.],
       [0., 3., 1., 1.]])

In [40]:
cnb = CategoricalNB()
cnb.fit(X_q5_std, y_q5_train)

CategoricalNB(alpha=1.0, class_prior=None, fit_prior=True)

In [42]:
print(f"Train score: {cnb.score(X_q5_train_std, y_q5_train)}")
print(f"Test score: {cnb.score(X_q5_test_std, y_q5_test)}")

Train score: 0.792
Test score: 0.784


## Questão 4

Considerando os dados presentes no arquivo classificacao_Q4.csv, treine o algoritmo k-Nearest Neighbors (KNN com k=15 e distância Euclidiana) utilizando a metodologia de validação cruzada k-fold com 10 folds não estratificados. Considere que a primeira pasta de validação seja formada pelas primeiras 10% linhas do arquivo, que a segunda pasta de validação seja formada pelas 10% linhas seguintes, e assim por diante, até atingir a última pasta, formada pelas 10% linhas finais da base. Qual o valor médio da acurácia para as bases de validação?

In [52]:
q6 = pd.read_csv('data/asset-v1_IRONHACK+DAFT+202002_SAO+type@asset+block@classificacao_Q4.csv')
q6.head(3)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,target
0,1.696199,-0.792598,-0.349427,-0.464560,3.187014,0.035976,1.033274,-1.504968,0.204693,1.691204,...,1.488142,-0.686337,2.084970,-0.685140,-2.049451,2.015426,1.158477,-0.309441,-1.549833,4.0
1,-0.236696,-2.202342,0.024023,1.497700,-0.069758,-2.467088,1.126529,-0.570557,2.079251,-1.882632,...,0.405567,0.509564,1.374071,-0.016943,-0.429280,-0.895016,1.259566,-0.354139,0.806797,5.0
2,-0.436683,1.563816,-0.895999,-0.580425,0.311060,-0.187369,0.805249,-2.399522,-0.578818,1.586981,...,0.933578,-1.285978,0.503162,0.204829,-0.753835,0.290033,1.721487,1.304518,0.478903,3.0


In [53]:
X_q6 = q6.drop(['target'], axis=1)
y_q6 = q6['target']

In [67]:
## precisa standarizar o X para KNN
X_q6_std = StandardScaler().fit_transform(X_q6)
knn_q6 = KNeighborsClassifier(n_neighbors=15, p=2, metric='minkowski')
result_q6 = cross_validate(knn_q6, X_q6_std, y_q6, cv=KFold(10), return_train_score=True)

In [68]:
print(f"Mean Score with KFold: {result_q6['train_score'].mean()}")
print(f"Mean Score with KFold: {result_q6['test_score'].mean()}")

Mean Score with KFold: 0.6597777777777778
Mean Score with KFold: 0.536


## Questão 5) 

Assinale as alternativas com V ou F para Verdadeiro ou Falso respectivamente. Atente para o fato que uma questão errada anula uma certa. Em caso de dúvidas deixe em branco.

(**V**) Árvores de decisão podem ser utilizadas para categorizar variáveis e, também, para descobrir regras que mostram interações entre variáveis.

(**F**) As folhas de uma árvore de classificação buscam minimizar a pureza entre classes.

(**V**) Uma árvore de decisão completa (profundidade máxima possível) tem maior chance de fazer overfitting nos dados do que uma árvore com profundidade limitada.

(**V**) O método KNN usualmente não é apropriado para base com muitos atributos (variáveis).

( ) Uma rede neural com uma camada intermediária com n neurônios, em que n é o número de atributos ou colunas, é um aproximador universal.

(**F**) O grau do polinômio do algoritmo Support Vector Machine (SVM) deve ser definido para todos os kernels.

(**V**) Algoritmos baseados em Boosting sequencialmente induzem modelos de tal forma que cada modelo gerado procura corrigir os erros cometidos pelos modelos anteriores.

(**F**) O Root Mean Squared Error (RMSE) é uma medida adequada para avaliar o desempenho final de um classificador.

(**V**) Recall e F1-score são métricas apropriadas para avaliar problemas de classificação em que se observa um desbalanceamento entre as classes.

(**V**) A metodologia de validação Leave-One-Out pode ser utilizada para obter melhor desempenho do modelo pois treina o modelo com a maior quantidade de dados possível.